In [ ]:
#!pip install threadpoolctl

In [1]:
import pandas as pd
import numpy as np
from joblib import load

In [2]:
# Inputs
path_g2022_vars = "g2022_adid_01062021_11082022_text.csv.gz"
# this is the output table from part of data-post-production repo that merges preprocessed results.
# Source: data-post-production/01-merge-results/01_merge_preprocessed_results


In [3]:
df = pd.read_csv(path_g2022_vars)

In [4]:
df

,ad_id,wmp_creative_id,ad_type,ad_title,google_asr_text,aws_ocr_video_text,aws_ocr_img_text,csum_agg,advertiser_id,advertiser_name,ad_text
0,CR00000257354440376321,cid_16941,VIDEO,NaN,Democratic candidate for governor. Stacey Abra...,» RECORDED\nFOX31\nSTACEYABRAMS\nGOVERNOR\nSpe...,NaN,3a0b45af177cfef8675852003ebb1e57838275d50d20b6...,AR08588079303567081473,Abrams For Governor,NaN
1,CR00000354386341527553,cid_1742,TEXT,Colorado Proposition GG,NaN,NaN,NaN,NaN,AR03715945093920718849,PROGRESSNOW COLORADO,Learn About Colorado Proposition GG Concerning...
2,CR00000387990165651457,cid_3457,TEXT,Fight for Better Schools,NaN,NaN,NaN,NaN,AR02250229023886016513,Mo Kashmiri for Sacramento County Office of Ed...,Mo Kashmiri for Sacramento County Board of Edu...
3,CR00000440354406924289,cid_39402,IMAGE,NaN,NaN,NaN,"Better Jobs, Higher Wages, Yang York For New F...",NaN,AR11849446020951834625,YANG FOR NEW YORK INC,NaN
4,CR00000501995777556481,cid_38087,IMAGE,NaN,NaN,NaN,2 9 10 8 7 11 1.2 6 5 1 4 3,NaN,AR01324533691786985473,KEVIN COMBS,1 Message Pending\nView Pending Messages\nNow
...,...,...,...,...,...,...,...,...,...,...,...
179151,CR07194247036013641729,cid_4339,VIDEO,How Much Energy Will the World Need?,We're headed toward an exciting. All renewable...,MATALO\nRARE EARTH\nPragerU;TWO DECADES\n+\nPr...,NaN,a9430b5a86efa75ef2afe22d2389a6a858e242806c306f...,AR16784630625381384193,PRAGER UNIVERSITY FOUNDATION,NaN
179152,CR04756463662258978817,cid_12037,VIDEO,We Can't Afford To Be Silent,One of my favorite lines on the couch on the a...,WE CAN'T AFFORD\nTO BE SILENT\nTURNING POINT U...,NaN,ea8f4e6d7ef123bdc508c39cb30aceabff3d347e42096c...,AR12552007562821632001,"TURNING POINT USA, NFP",NaN
179153,CR12343661791669125121,cid_63,VIDEO,4-How to Beat Big Money,I've heard good things about Connecticut. So t...,FROM PAGE ONE\nRell Keeps Focus On Reform\nPAG...,NaN,nan 604ebf1b6d00ec3ec53177baa6bbe3e74fcbecda20...,AR16131524841651044353,RECLAIMTHEAMERICANDREAM ORG,NaN
179154,CR13232244551881588737,cid_7284,VIDEO,Not LeesBurk Ad: Inflation,"This is our town of Leesburg, not Leesburg. Ho...",SUZANNE FOX VOTED AGAINST IT\nSUZANNE D. FOX\n...,NaN,a5154514917f57ab4e8243301c9d5576656b6197babb76...,AR11905426212371038209,Suzanne Fox for Leesburg Mayor,NaN


In [5]:
# Replace newline characters with spaces
df['aws_ocr_video_text'] = df['aws_ocr_video_text'].str.replace('\n', ' ')
df['ad_text'] = df['ad_text'].str.replace('\n', ' ')
df['aws_ocr_img_text'] = df['aws_ocr_img_text'].str.replace('\n', ' ')
df['ad_title'] = df['ad_title'].str.replace('\n', ' ')
df['google_asr_text'] = df['google_asr_text'].str.replace('\n', ' ')

In [6]:
cols = ['advertiser_name', 'ad_title', 'google_asr_text', 'aws_ocr_img_text', 
        'aws_ocr_video_text', 'ad_text']
df['combined'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df['combined'] = df['combined'].str.strip()

In [7]:
df.shape

(179156, 12)

In [8]:
len(df.advertiser_name.unique())
#len(df.wmp_creative_id.unique())

5134

In [20]:
df_txt = df.groupby(['advertiser_id'])['combined'].apply(lambda x: ' '.join(x)).reset_index()

In [21]:
df_txt

,advertiser_id,combined
0,AR00000475401340059649,VINCE LEACH FOR SENATE nan nan A CHAMPION FOR ...
1,AR00008638175664668673,TINA SMITH FOR MINNESOTA Protect Our Majority ...
2,AR00013006535361822721,NICK LALOTA FOR CONGRESS Join Nick LaLota nan ...
3,AR00015213186479292417,Tom Lackey For Assembly 2012 nan nan PARKING -...
4,AR00017645374919409665,"Democratic Party of Oregon nan Oil companies, ..."
...,...,...
5143,AR18437367850864738305,AMERICANS FOR PROSPERITY nan Farm started in 1...
5144,AR18437938703557984257,Vote No On Amendment 1 Workers Rights Amendmen...
5145,AR18440333439883280385,Kane County Democratic Central Cmte Vote for K...
5146,AR18441871450492108801,SAM GRAVES VICTORY FUND Elect Sam Graves nan n...


In [11]:
# Load the best model weights
mnb_clf = load('models/party_clf_pdid_rf.joblib')

In [22]:
pred = mnb_clf.predict(df_txt['combined'])

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done 500 out of 500 | elapsed:    0.4s finished


In [23]:
df_txt['party_all_clf'] = pred

In [24]:
df_txt = df_txt[['advertiser_id', 'party_all_clf']]

In [25]:
df_txt

,advertiser_id,party_all_clf
0,AR00000475401340059649,REP
1,AR00008638175664668673,DEM
2,AR00013006535361822721,REP
3,AR00015213186479292417,REP
4,AR00017645374919409665,DEM
...,...,...
5143,AR18437367850864738305,REP
5144,AR18437938703557984257,REP
5145,AR18440333439883280385,DEM
5146,AR18441871450492108801,REP


In [26]:
df_txt.party_all_clf.value_counts()

party_all_clf
DEM      2966
REP      2180
OTHER       2
Name: count, dtype: int64

In [27]:
df_txt.to_csv("party_all_clf_google_2022_advertiser_id.csv", index=False)